In [1]:
# install libraries
!pip install groq gtts SpeechRecognition pydub pdfplumber sentence-transformers
!apt-get install -y tesseract-ocr
!pip install pytesseract


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.3/137.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 81.0 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.3.1
    Uninstalling click-8.3.1:
      Successfully uninstalled click-8.3.1
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.

In [2]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 26.3 MB/s eta 0:00:00


In [ ]:
from groq import Groq
from gtts import gTTS
from IPython.display import Audio
import speech_recognition as sr
from pydub import AudioSegment
from google.colab import files, drive
from PIL import Image
import pytesseract
import pdfplumber
import os
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import re


drive.mount('/content/drive')


api = "Your_Groq_Api_k"
client = Groq(api_key=api)


ncert_pdfs = [
    "/content/drive/MyDrive/10.pdf",
    "/content/drive/MyDrive/11.pdf",
    "/content/drive/MyDrive/12.pdf"
]


indexDir = "/content/drive/MyDrive/NCERT_INDEX"
os.makedirs(indexDir, exist_ok=True)
indexPath = os.path.join(indexDir, "ncert_faiss.index")
metaPath = os.path.join(indexDir, "ncert_chunks.npy")

Mounted at /content/drive


In [5]:
braille = {
    'a':'⠁','b':'⠃','c':'⠉','d':'⠙','e':'⠑',
    'f':'⠋','g':'⠛','h':'⠓','i':'⠊','j':'⠚',
    'k':'⠅','l':'⠇','m':'⠍','n':'⠝','o':'⠕',
    'p':'⠏','q':'⠟','r':'⠗','s':'⠎','t':'⠞',
    'u':'⠥','v':'⠧','w':'⠺','x':'⠭','y':'⠽','z':'⠵',

    ' ':' ',

    '#':'⠼',
    '0':'⠚','1':'⠁','2':'⠃','3':'⠉','4':'⠙','5':'⠑',
    '6':'⠋','7':'⠛','8':'⠓','9':'⠊',

    ',':'⠂',';':'⠆',':':'⠒','.' :'⠲','?':'⠦','!':'⠖',
    '\'':'⠄','-':'⠤',

    '"':'⠶','“':'⠘⠦','”':'⠘⠴','‘':'⠄⠦','’':'⠄⠴',

    '(':'⠐⠣',')':'⠐⠜',

    '/':'⠸⠌','\\':'⠸⠡',

    '–':'⠠⠤','—':'⠐⠠⠤',

    '+':'⠖','=':'⠶','*':'⠔','%':'⠨⠴',

    'letter_sign':'⠰'
}

def brailleLine(line):
    return ''.join(braille.get(ch.lower(), '?') for ch in line)


In [6]:
def voiceToText(audFile):
    recg = sr.Recognizer()

    sund = AudioSegment.from_file(audFile)
    wav = "converted.wav"
    sund.export(wav, format="wav")

    with sr.AudioFile(wav) as source:
        aud = recg.record(source)

    try:
        return recg.recognize_google(aud)
    except:
        return "Voice Error"

#
def imgToText(imgFile):
    return pytesseract.image_to_string(Image.open(imgFile))

In [7]:
def ai(ques):
    prompt = f"Solve step-by-step and explain clearly:\n{ques}"

    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [8]:
def brailleAccuracy(text):
    total = 0
    correct = 0
    for ch in text.lower():
        if ch == "\n":
            continue
        total += 1
        if ch in braille:
            correct += 1
    if total == 0:
        return 0
    return round((correct / total) * 100, 2)

def ocrAccuracy(text):
    total = 0
    valid = 0
    for ch in text:
        if ch.strip() == "":
            continue
        total += 1
        if ch.isalnum() or ch in ",.;:?!()[]{}+-=*/%\\":
            valid += 1
    if total == 0:
        return 0
    return round((valid / total) * 100, 2)

In [9]:
print("Choose the input \n")
print("1 for Type Question")
print("2 for voice uplode")
print("3 for image uplode")

uplodes = input("Enter input type : ")

if uplodes == "1":
    question = input("Enter the quesiton \n ")

elif uplodes == "2":
    print("uplode the voice file \n")
    uplode = files.upload()
    files = list(uplode.keys())[0]
    question = voiceToText(files)

elif uplodes == "3":
    print("uplode the image")
    uplode = files.upload()
    files = list(uplode.keys())[0]
    question = imgToText(files)

    #
    ocr_acc = ocrAccuracy(question)
    print("\nOCR Accuracy:", ocr_acc, "%\n")

else:
    print("Invalid input")
    raise SystemExit

Choose the input 

1 for Type Question
2 for voice uplode
3 for image uplode
Enter input type : 1
Enter the quesiton 
 x+2=3


In [10]:
answer = ai(question)

#this is some extra
braille_acc = brailleAccuracy(answer)
print("Braille Accuracy:", braille_acc, "%\n")

print("Solution In English")
print(answer)

print("Solution In Brainlle \n")
for line in answer.split("\n"):
    print(brailleLine(line))


tts = gTTS(text=answer, lang="en", tld="co.in", slow=True)
tts.save("answer.mp3")

print("Solutin in audio formate \n")
Audio("answer.mp3", autoplay=True)

Braille Accuracy: 100.0 %

Solution In English
To solve the equation x + 2 = 3, we need to find the value of x. Here are the steps:

**Step 1: Write down the equation**
The equation is: x + 2 = 3

**Step 2: Isolate the variable (x)**
To isolate x, we need to get rid of the +2 on the left side of the equation. We can do this by subtracting 2 from both sides of the equation. This is because whatever we do to one side of the equation, we must also do to the other side to keep the equation balanced.

So, subtract 2 from both sides:
x + 2 - 2 = 3 - 2

**Step 3: Simplify the equation**
Now, simplify the equation by combining the constants on both sides:
x = 1

Therefore, the value of x is 1.

**Check:**
Let's plug x = 1 back into the original equation to make sure it's true:
1 + 2 = 3
3 = 3

Yes, it's true! So, the solution to the equation x + 2 = 3 is x = 1.
Solution In Brainlle 

⠞⠕ ⠎⠕⠇⠧⠑ ⠞⠓⠑ ⠑⠟⠥⠁⠞⠊⠕⠝ ⠭ ⠖ ⠃ ⠶ ⠉⠂ ⠺⠑ ⠝⠑⠑⠙ ⠞⠕ ⠋⠊⠝⠙ ⠞⠓⠑ ⠧⠁⠇⠥⠑ ⠕⠋ ⠭⠲ ⠓⠑⠗⠑ ⠁⠗⠑ ⠞⠓⠑ ⠎⠞⠑⠏⠎⠒

⠔⠔⠎⠞⠑⠏ ⠁⠒ ⠺⠗⠊⠞⠑ ⠙⠕⠺⠝ ⠞⠓⠑ 